In [1]:
#04/02/2019
import numpy as np # Algebre linéaire
import pandas as pd # Data processing, CSV

from PIL import Image # Image display
import matplotlib.pyplot as plt

from io import BytesIO # Image request
import requests

import bq_helper # Data train querry
from bq_helper import BigQueryHelper

import os

In [2]:
def images_from_url(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

In [3]:
open_images = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="open_images")
bq_assistant = BigQueryHelper("bigquery-public-data", "open_images")

# Liste des tables :
#['annotations_bbox', 'dict', 'images', 'labels']

Using Kaggle's public dataset BigQuery integration.
Using Kaggle's public dataset BigQuery integration.


In [9]:
subQueryImages = """
(SELECT image_id, original_url
FROM `bigquery-public-data.open_images.images`)
"""

subQueryBox = """
(SELECT image_id, label_name, x_min, x_max, y_min, y_max
FROM `bigquery-public-data.open_images.annotations_bbox`
ORDER BY image_id
LIMIT 100000)
"""

subQueryWord = """
(SELECT label_name, label_display_name
FROM `bigquery-public-data.open_images.dict`)
"""

mainQuery = """
SELECT img.image_id, img.original_url, wrd.label_display_name, box.label_name, box.x_min, box.x_max, box.y_min, box.y_max
FROM """ + subQueryImages + """ img
INNER JOIN """ + subQueryBox + """ box ON img.image_id = box.image_id
INNER JOIN """ + subQueryWord + """wrd ON box.label_name = wrd.label_name
ORDER BY img.image_id
"""

labelQuery = """
SELECT ROW_NUMBER() OVER (ORDER BY wrd.label_name) - 1 AS row, wrd.label_name, wrd.label_display_name
FROM (SELECT DISTINCT(label_name) FROM """ + subQueryBox + """) lab
JOIN """ + subQueryWord + """ wrd ON lab.label_name = wrd.label_name"""



dataset = open_images.query_to_pandas_safe(mainQuery)
labelUnique = open_images.query_to_pandas_safe(labelQuery)

"""
for i in range(0, 10):
    plt.imshow(images_from_url(dataset.original_url[i]))
    plt.show()
    print(dataset.label_display_name[i])
    print(dataset.original_url[i])"""

'\nfor i in range(0, 10):\n    plt.imshow(images_from_url(dataset.original_url[i]))\n    plt.show()\n    print(dataset.label_display_name[i])\n    print(dataset.original_url[i])'

In [10]:
print(labelUnique)

     row   label_name label_display_name
0      0    /m/011k07           Tortoise
1      1  /m/011q46kg          Container
2      2    /m/012074             Magpie
3      3    /m/0120dh         Sea turtle
4      4    /m/01226z           Football
5      5    /m/012n7d          Ambulance
6      6    /m/012w5l             Ladder
7      7    /m/012xff         Toothbrush
8      8    /m/012ysf            Syringe
9      9    /m/0130jx               Sink
10    10    /m/0138tl                Toy
11    11    /m/013y1f              Organ
12    12    /m/01432t      Cassette deck
13    13    /m/014j1m              Apple
14    14    /m/014sv8                Eye
15    15    /m/014trl          Cosmetics
16    16    /m/014y4n             Paddle
17    17    /m/0152hh            Snowman
18    18     /m/01599               Beer
19    19    /m/015h_t              Beard
20    20     /m/015p6               Bird
21    21    /m/015qbp      Parking meter
22    22    /m/015qff      Traffic light
23    23    /m/0

In [ ]:
"""
file = images_from_url(dataset.original_url[120])

tmp = np.array(file.resize((100, 100)))
plt.imshow(tmp)
plt.show()
print(dataset.label_display_name[120])
"""

features = []
labels = []

# dataset.label_name.size
data_length = 10

for data_ex in range (0, data_length):
    features.append(np.array(images_from_url(dataset.original_url[data_ex]).resize((100, 100))))
    labels.append([[dataset.label_name[data_ex]],[dataset.x_min[data_ex], dataset.y_min[data_ex], dataset.x_max[data_ex], dataset.y_max[data_ex]]])

In [ ]:
for tmp in range(0, 10):
    plt.imshow(features[tmp])
    plt.show()
    print(labels[tmp])